<a href="https://colab.research.google.com/github/catasaurus/random-ai-notebooks/blob/main/bart_finetune_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets
from transformers import TFAutoModel, BartTokenizer, DefaultDataCollator
from datasets import load_dataset, load_metric, Dataset, DatasetDict
from tqdm import tqdm
import tensorflow as tf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
paws = load_dataset('paws', 'labeled_final')
paws

Reusing dataset paws (/root/.cache/huggingface/datasets/paws/labeled_final/1.1.0/8d567c6472623f42bd2cc635cad06932d0f0cd2f897db56013c1180f4317d338)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 49401
    })
    test: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 8000
    })
    validation: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 8000
    })
})

In [3]:
import csv
from pathlib import Path

colnames = 'sentence1 sentence2 label'.split()
filepath = Path('_'.join(colnames)).with_suffix('.csv')

with filepath.open('w') as fout:
    writer = csv.DictWriter(fout, fieldnames=colnames)
    writer.writeheader()
    for arrows in [paws['train'], paws['test'], paws['validation']]:
        for row in arrows:
            writer.writerow({name: row[name] for name in colnames})

In [4]:
quora = load_dataset('quora')

Using custom data configuration default
Reusing dataset quora (/root/.cache/huggingface/datasets/quora/default/0.0.0/36ba4cd42107f051a158016f1bea6ae3f4685c5df843529108a54e42d86c1e04)


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
import pandas as pd
new_quora = {'sentence1':i['questions']['text'][0] for i in quora['train']}
new_quora['sentence2'] = [i['questions']['text'][1] for i in quora['train']]
new_quora['label'] = [1 if i['is_duplicate'] else 0 for i in quora['train']]
pd.DataFrame(new_quora).to_csv('quora_format_paws.csv')

In [6]:
paws = pd.read_csv('sentence1_sentence2_label.csv')
quora = pd.read_csv('quora_format_paws.csv')
df = pd.concat([paws, quora])
del paws
del quora
df.to_csv('data.csv')

In [7]:
df = pd.read_csv('data.csv').loc[:, ['sentence1', 'sentence2']]
x = df['sentence1'].to_numpy()
y = df['sentence2'].apply(lambda i: str(i)).to_numpy()
del df

In [8]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
all = tokenizer(list(x)+list(y), return_tensors='tf', padding=True)['input_ids']
split = tf.split(all, 2, axis=0)
del all
x = split[0]
y = split[1]

In [9]:
model = TFAutoModel.from_pretrained("facebook/bart-large", forced_bos_token_id=0)

All model checkpoint layers were used when initializing TFBartModel.

All the layers of TFBartModel were initialized from the model checkpoint at facebook/bart-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartModel for predictions without further training.


In [10]:
model.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [ ]:
model.fit(x, y, epochs=10)
model.save('finetuned_bart.h5')

Epoch 1/10
